In [1]:
import pandas as pd
from scipy.stats import friedmanchisquare
import csv

In [2]:
csv_url = 'https://media.githubusercontent.com/media/Sank6/S-Unit-Attacks/refs/heads/main/experiment_4/s_unit_attack_output_2025-04-18_11.02.32.csv'
df = pd.read_csv(csv_url)

In [3]:
df

,n,k,start_index,end_index,iteration,num_inner_iteration,alpha_in,alpha_in_norm,alpha_out,alpha_out_norm,time
0,7,1,0,1,1,2,4*x^5 + x^4 + x^3 - x^2 + 2*x + 2,5.196152,5/2*x^5 + 3/2*x^4 + 5/2*x^3 + x^2 + 3*x + 1/2,5.000000,0.256962
1,7,1,0,1,2,2,-3*x^5 - 3*x^4 - 2*x^3 + x^2 - x - 1,5.000000,-x^5 - 3/2*x^4 - 2*x^3 - 3/2*x^2 - 7/2*x - 3/2,4.898979,0.016421
2,7,1,0,1,3,2,x^5 + 3*x^4 + x^3 - 13*x^2 - x,13.453624,-11/4*x^5 - 17/4*x^3 - 17/4*x^2 + 11/4*x + 1/2,7.176350,0.014914
3,7,1,0,1,4,2,22*x^4 + 3*x^3 - 12*x^2 + 22*x + 42,53.712196,-23/4*x^5 + 19/2*x^4 - 11/4*x^3 + 23/4*x^2 + 3...,15.435349,0.015118
4,7,1,0,1,5,2,349*x^5 - 2*x^4 - x^3 + x^2 + x + 1,349.011461,-1047/16*x^5 - 523/8*x^4 - 345/16*x^3 - 3/16*x...,115.659522,0.016350
...,...,...,...,...,...,...,...,...,...,...,...
434104,12,23,1,24,496,2,x^3 + x^2 - 2*x - 1,2.645751,x^3 - x,1.414214,0.765410
434105,12,23,1,24,497,2,-x^3 + 2*x^2 - 2*x,3.000000,x^3,1.000000,0.759246
434106,12,23,1,24,498,1,-3*x^3 - x^2 + 8*x + 2,8.831761,-3*x^3 - x^2 + 8*x + 2,8.831761,0.403047
434107,12,23,1,24,499,2,4*x^2 + 6*x,7.211103,2*x,2.000000,0.729528


In [5]:
significance_level = 0.05
grouped_data = df.groupby(['n', 'k'])

titles = ['n', 'k', 'chi_squared', 'p_value', 'num_blocks', 'num_treatments', 'significant']

csvfile = open('h_test.csv', 'w', newline='')
writer = csv.DictWriter(csvfile, fieldnames=titles)
writer.writeheader()

for (n, k), group in grouped_data:
    pivot_table = group.pivot(index='iteration', columns='start_index', values='alpha_out_norm')

    num_blocks = pivot_table.shape[0]
    num_treatments = pivot_table.shape[1]

    if num_treatments <= 2: continue
    
    treatments_data = [pivot_table[c].values for c in pivot_table.columns]
    
    chi_squared_stat, p_value = friedmanchisquare(*treatments_data)
    writer.writerow({
        'n': n,
        'k': k,
        'chi_squared': chi_squared_stat,
        'p_value': p_value,
        'num_blocks': num_blocks,
        'num_treatments': num_treatments,
        'significant': p_value < significance_level
    })

csvfile.close()